## TYK2 FlowDMS Offsets

For the most recent full TYK2 FlowDMS, we obtained conflicting summary statistics from different runs. The underlying reason is that, for one run, the chunks were separated before processing and in another they were not. This does not matter to the model itself, which operates per-position and only uses WT counts from the same chunk. However, it _does_ matter for computing the _offset_, which is taken as the `mean(log(count))` within each sample. This quantity was only computed within each sample-chunk when the chunks were pre-separated, but within each sample otherwise.

To see how this leads to the effect we observe in the midpoints, let's consider several models who differ only in the offset:

  - `mean(log(count))`
  - `log(sum(stop_counts))`
  - `log(sum(all_counts))`
  - no offset

As an example, let's grab a position chunk 2 and do these regressions, pull out the WT marginals, and compute the midpoints.

In [101]:
library(data.table)
library(emmeans)
library(furrr)
library(future)
library(glmmTMB)
library(tidyverse)

In [287]:
run_model <- function(dataset){
    formula <- as.formula(count ~ -1 + condition_conc*mut_aa + (1 | barcode) + (1 | sample) + offset(offset_col))

    mod <- glmmTMB(formula = formula,
            REML = TRUE,
            start = -1,
            control = glmmTMBControl(optimizer = optim,
                 optArgs = list(method = "L-BFGS-B",
                    pgtol = 0,
                    rel.tol = 0.1)),
            data = dataset,
            family = nbinom2)
    
    coefs <- broom.mixed::tidy(mod) %>%
        unnest_longer(term) %>%
        mutate(dispersion = sigma(mod))

    marginals <- broom::tidy(emmeans(mod, ~mut_aa + condition_conc))

    return_data <- bind_rows(coefs, marginals)

    return(return_data)
}

In [208]:
mapped_counts <- data.table::fread("../../dms/pipeline/OCNT-VAMPLIB-1-assay-run2-all-assigned-split.mapped-counts.tsv",
        col.names = c("sample", "barcode", "count", "lib",
                      "chunk", "wt_aa", "pos", "mut_aa",
                      "wt_codon", "mut_codon", "chunkid", "dox",
                      "condition", "condition_conc", "clone")) %>%
    mutate(condition_conc = as.factor(condition_conc),
        mut_aa = if_else(wt_aa == mut_aa | is.na(mut_aa), "WT", mut_aa),
        mut_aa = relevel(as.factor(mut_aa), ref = "WT")) %>%
    ungroup() %>%
    group_by(condition_conc) %>%
    mutate(log_stop_counts_sample = log(sum(count[which(mut_aa %in% c("*", "X", "Stop", "stop"))])),
        mean_log_count_sample = mean(log(count)),
        log_total_count_sample = log(sum(count))) %>%
    ungroup() %>%
    group_by(condition_conc, chunk) %>%
    mutate(log_stop_counts_chunk = log(sum(count[which(mut_aa %in% c("*", "X", "Stop", "stop"))])),
        mean_log_count_chunk = mean(log(count)),
        log_total_count_chunk = log(sum(count)))

In [209]:
pos669 <- mapped_counts %>%
    filter(pos == 669 | (mut_aa == "WT" & chunk == 10)) %>%
    mutate(condition_conc = as.factor(condition_conc),
           mut_aa = relevel(as.factor(mut_aa), ref = "WT"))

In [290]:
mod_offset <- run_model(pos669 %>% mutate("offset_col" = 0))
mod_offset %>%
    filter(mut_aa == "WT") %>%
    dplyr::select(estimate, mut_aa, condition_conc) %>%
    mutate(condition_conc = as.numeric(condition_conc) / 100) %>%
    mutate(estimate = (estimate - min(estimate))/sum(estimate - min(estimate))) %>%
    summarize(mp = sum((condition_conc - 0.125)*estimate))

In [298]:
mod_offset %>%
    filter(is.na(term)) %>%
    dplyr::select(estimate, mut_aa, condition_conc) %>%
    group_by(mut_aa) %>%
    mutate(condition_conc = as.numeric(condition_conc) / 100) %>%
    mutate(estimate = (estimate - min(estimate))/sum(estimate - min(estimate))) %>%
    summarize(mp = sum((condition_conc - 0.125)*estimate))

mut_aa,mp
<chr>,<dbl>
*,0.3085959
A,0.2909311
C,0.2519957
D,0.2786156
E,0.2641033
F,0.3174586
G,0.2402088
I,0.2591580
K,0.3164403


In [164]:
container_list <- as.list(rep(NA, 6))
names(container_list) <- names(pos669)[15:20]

In [58]:
for(i in 1:length(container_list)){
    container_list[[i]] <- run_model(pos669 %>% rename("offset_col" = names(container_list)[i]))
}

bind_rows(container_list, .id = "offset") %>%
    filter(mut_aa == "WT") %>%
    dplyr::select(offset, estimate, mut_aa, condition_conc) %>%
    group_by(offset) %>%
    mutate(condition_conc = as.numeric(condition_conc) / 100) %>%
    mutate(estimate = (estimate - min(estimate))/sum(estimate - min(estimate))) %>%
    summarize(mp = sum((condition_conc - 0.125)*estimate))

NOTE: A nesting structure was detected in the fitted model:
    mut_aa %in% condition_conc

NOTE: A nesting structure was detected in the fitted model:
    mut_aa %in% condition_conc

NOTE: A nesting structure was detected in the fitted model:
    mut_aa %in% condition_conc

NOTE: A nesting structure was detected in the fitted model:
    mut_aa %in% condition_conc

NOTE: A nesting structure was detected in the fitted model:
    mut_aa %in% condition_conc

NOTE: A nesting structure was detected in the fitted model:
    mut_aa %in% condition_conc



In [278]:
raw_sumstats <- read_tsv("../../bms-dms/sumstats/TYK2-VAMP/run2B/OCNT-VAMPLIB-1-assay-run2B-vampseq.sumstats.tsv")
sumstats <-  raw_sumstats %>%
    filter(grepl("aa",term), pos != 1188) %>%
    separate(term, c("condition", "aa"), ":") %>%
    mutate(estimate = estimate / log(2),
           std.error = std.error / log(2),
           condition = gsub("condition_conc", "", condition),
           aa = gsub("mut_aa", "", aa),
           aa = if_else(aa %in% c("*","X"), "*", aa))

marginals <- raw_sumstats %>% filter(is.na(term))
weights <- marginals %>%
    dplyr::select(condition_conc, chunk, pos, mut_aa, estimate, std.error) %>%
    rename("bin" = "condition_conc") %>%
    group_by(chunk, pos, mut_aa) %>%
    mutate(bin = case_when(bin == 25 ~ 0.2,
                           bin == 50 ~ 0.4,
                           bin == 75 ~ 0.6,
                           TRUE ~ 0.8))

aa_uniq <- unique(sumstats$aa)
pos_uniq <- 1:1187

weights_nest <- weights %>% nest(data = c(-chunk, -pos, -mut_aa))
metadata_df <- weights_nest %>% dplyr::select(-data)

Rows: 200436 Columns: 16
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (7): chunk, effect, component, group, term, mut_aa, version
dbl (8): pos, estimate, std.error, statistic, p.value, dispersion, condition...
lgl (1): clone

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [283]:
plan(multicore, workers = 40)
source("../../dms/src/dms-analysis-utils.R")
midpoints <- future_map2_dfr(.x = weights_nest$data,
                             .y = transpose(metadata_df),
                             ~generate_resamples(.x$estimate,
                                                 .x$std.error,
                                                 .y,
                                                 num = 1000))

Warning message:
“The `x` argument of `as_tibble.matrix()` must have unique column names if
`.name_repair` is omitted as of tibble 2.0.0.
ℹ Using compatibility `.name_repair`.”
Warning message:
“UNRELIABLE VALUE: Future (‘<none>’) unexpectedly generated random numbers without specifying argument 'seed'. There is a risk that those random numbers are not statistically sound and the overall results might be invalid. To fix this, specify 'seed=TRUE'. This ensures that proper, parallel-safe random numbers are produced via the L'Ecuyer-CMRG method. To disable this check, use 'seed=NULL', or set option 'future.rng.onMisuse' to "ignore".”
Warning message:
“The `x` argument of `as_tibble.matrix()` must have unique column names if
`.name_repair` is omitted as of tibble 2.0.0.
ℹ Using compatibility `.name_repair`.”
Warning message:
“UNRELIABLE VALUE: Future (‘<none>’) unexpectedly generated random numbers without specifying argument 'seed'. There is a risk that those random numbers are not statis

In [284]:
wt_scores <- midpoints %>%
    filter(mut_aa == "WT") %>% 
    group_by(chunk) %>%
    summarize("WT score" = median(score_mean),
              "WT score standard error" = median(score_se))
wt_scores

chunk,WT score,WT score standard error
<chr>,<dbl>,<dbl>
1,0.5136863,0.0005450210
10,0.5148691,0.0007092281
11,0.5132871,0.0005595076
12rc,0.5148232,0.0004850871
13,0.5100740,0.0006218095
14,0.5116382,0.0006462137
15,0.5089319,0.0006772927
16,0.5113239,0.0006617800
17,0.5115541,0.0006315081


In [285]:
midpoints_test <- midpoints %>%
    filter(mut_aa != "WT") %>%
    left_join(wt_scores, by = "chunk") %>%
    mutate(estimate = score_mean - `WT score`,
           std.error = sqrt(score_se^2 + `WT score standard error`^2),
           statistic = estimate/std.error,
           p.value = pmin(pnorm(statistic, mean = 0, sd = 1)*2,
                                 (1-pnorm(statistic, sd = 1))*2),
           p.adj = p.adjust(p.value, method = "BH"))

In [286]:
midpoints_test %>%
    summarize(median(statistic))

median(statistic)
<dbl>
-0.568071


In [270]:
p <- read_tsv("../../bms-dms/sumstats/TYK2-VAMP/run2B/OCNT-VAMPLIB-1-assay-run2B-vampseq-midpoints.sumstats.tsv")
p %>%
    group_by(chunk) %>%
    summarize(median(statistic))

Rows: 23718 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (2): chunk, mut_aa
dbl (10): score_mean, score_se, pos, WT score, WT score standard error, esti...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


chunk,median(statistic)
<chr>,<dbl>
1,-0.3468803
10,-1.6403115
11,-1.1583840
12rc,-0.8117957
13,-0.8169084
14,-1.2054380
15,-2.5963509
16,-1.1329588
17,-1.0344762
